# Session 3: Text analysis approaches

\#\#\# __DRAFT__ \#\#\#

Text analysis is a classic computational and data science problem, 

Compared with regression and classification approaches on continuous and categorical dataset taking text data and deriving distinct insights is a far more complicated task. Text data and especially free text (text fields in sentence form) is typically classed as a form of unstructured data because of the various nuances introduced by languages.

With the ever increasing computational power has come a side-by-side improvements in approaches to text analysis. 

The idea of topic modelling, identifying abstract 'topics' within a collection of documents (corpus) using statistical models, was first described in 1998, with probabilistic latent semantic analysis (PLSA) outlined in 1999 and latent Dirichlet allocation (LDA) developed in [2002](http://jmlr.csail.mit.edu/papers/v3/blei03a.html). LDA has become one of the most commonly used topic modelling approaches since although many extensions of LDA have since been proposed.

In [ ]:
import numpy as np
import pandas as pd
import gensim
import matplotlib.pyplot as plt

In [ ]:
%%bash 

if test -f data/subset_twcs.csv; then
    echo "Data file exists"
else 
    tar xzf data/subtwcs.tar.gz
fi

In [ ]:
# import the dataset

twitter_data = pd.read_csv('data/subset_twcs.csv')

twitter_data.head()

In [ ]:
twitter_data.shape

In [ ]:
# create a sub dataframe for just data that was inbound
inbound_dat = twitter_data[twitter_data.inbound == True]

inbound_dat.head()

In [ ]:
inbound_dat.author_id.unique().shape

In [ ]:
# join together all inbound tweets from the same user

user_tweets = inbound_dat.groupby('author_id')['text'].apply(lambda x: ','.join(x))

In [ ]:
user_tweets = user_tweets.reset_index()

user_tweets.text[:10].tolist()

## Preprocessing

Pre processing is a crucial step in any text analytics project. Text data on its own is very difficult for machines to understand and therefore it requires cleaning and preparing before building models. This often involves a number of steps such as:
- Tokenisation, converting a long string of words into a list of individual words i.e. "the cat sat on the mat" -> ["the", "cat", "sat", "on", "the", "mat"]
- Noise removal, most commonly removing punctuation or things like hyperlinks or emojis
- Stopword removal, removing common words that don't contain information such as the, and, or, a 
- Stemming or lemming, this is the process of reverting words to their root either by chopping off suffixes (stemming) or reverting to word lemma (lemming)
- Normalisation, commonly this means converting all words to lower or uppercase


In [ ]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, remove_stopwords

def basic_preprocess(list_of_strings):
    """
    A basic function that takes a list of strings and runs some basic
    gensim preprocessing to tokenise each string.
    
    Operations:
        - convert to lowercase
        - remove html tags
        - remove punctuation
        - remove numbers
    
    Outputs a list of lists
    """
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_numeric, remove_stopwords]

    preproc_text = [preprocess_string(doc, CUSTOM_FILTERS) for doc in list_of_strings]
    
    return preproc_text

In [ ]:
# what are stop words?

from gensim.parsing.preprocessing import STOPWORDS

print(STOPWORDS)

In [ ]:
import re

def remove_twitterisms(list_of_strings):
    """
    Some regular expression statements to remove twitter-isms
    
    Operations:
        - remove links
        - remove @tag
        - remove #tag
        
    Returns list of strings with the above removed
    """
    
    # removing some standard twitter-isms

    list_of_strings = [re.sub(r"http\S+", "", doc) for doc in list_of_strings]

    list_of_strings = [re.sub(r"@\S+", "", doc) for doc in list_of_strings]

    list_of_strings = [re.sub(r"#\S+", "", doc) for doc in list_of_strings]
    
    return list_of_strings

In [ ]:
# removing emojis
# taken from https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b#gistcomment-3315605

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
from gensim.models.phrases import Phrases

def n_gram(tokens):
    """Identifies common two/three word phrases using gensim module."""
    # Add bigrams and trigrams to docs (only ones that appear 10 times or more).
    # includes threshold kwarg (threshold score required by bigram)
    bigram = Phrases(tokens, min_count=10, threshold=100)
    trigram = Phrases(bigram[tokens], threshold = 100)

    for idx, val in enumerate(tokens):
        for token in bigram[tokens[idx]]:
            if '_' in token:
                if token not in tokens[idx]:
                    # Token is a bigram, add to document.bigram
                    tokens[idx].append(token)
        for token in trigram[tokens[idx]]:
            if '_' in token:
                if token not in tokens[idx]:
                    # Token is a trigram, add to document.
                    tokens[idx].append(token)
    return tokens

In [ ]:
import random

def test_basic_proprocess():
    
    # just to randomise tests
    randomize = random.randrange(0,3)
    
    test_doc = [
        "THE CAT SAT ON THE MAT!",
        "<h1> hello world?</h1>",
        "wElL WeLl We?L, wHaT <strong>HaVe wE</strong>hERe"
    ]
    
    test_output = basic_preprocess(test_doc)
    
    # test if func returns list of lists
    assert isinstance(test_output[randomize], list)
    # test if all items 
    assert [x.islower() for x in test_output[randomize]] == [True] * len(test_output[randomize])
    # test if html tags are removed
    assert "<h1>" not in test_output[1]

In [ ]:
# next we implement the preprocessing steps

preprocessed_corpus = remove_twitterisms(user_tweets.text.tolist())

preprocessed_corpus = [remove_emoji(doc) for doc in preprocessed_corpus]

preprocessed_corpus = basic_preprocess(preprocessed_corpus)

preprocessed_corpus = n_gram(preprocessed_corpus)

In [ ]:
# lets compare the original strings to the preprocessed strings
for orig, proc in zip(user_tweets.text.tolist()[:10], preprocessed_corpus[:10]):
    
    print(orig)
    print(proc)
    print('\n')

## Building a bag of words corpus and dictionary

In [ ]:
import numpy as np
from gensim.corpora import Dictionary

def bag_of_word_processing(corpus_of_tokens, lower_extreme, upper_extreme):
    """
    Take the list of tokens and convert them into a bag-of-words (BoW) format.

    Extended description of function.

    :param list of lists corpus_of_tokens: a list of strings produced during preprocessing representing all documents in corpus
    :param int lower_extreme: Description of arg2.
    :param float upper_extreme: the upper extreme filter limit, words are excluded if they occur in more documents than the proportion specified here
    :return: gensim.corpora.dictionary.Dictionary object 
    :return: list representing BoW corpus
    
    """

    # Create a dictionary representation of the documents.
    # gensim Dictionary function creates tokens -> tokenID dict
    dictionary = Dictionary(corpus_of_tokens)
    print('Number of unique words in initital documents:', len(dictionary))

    org_dict = len(dictionary)

    # Filter out words that occur less than 10 documents, or more than 70% of the documents.
    dictionary.filter_extremes(no_below=lower_extreme, no_above=upper_extreme)
    print('Number of unique words after removing rare and common words:', len(dictionary))

    filt_dict = len(dictionary)

    print('Token reduction of: ' + str((1-filt_dict/org_dict)*100)+'%')

    # transform to bag of words
    corpus = [dictionary.doc2bow(doc) for doc in corpus_of_tokens]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    # output on document length
    print('Average number of words per document before BoW transform: ', np.mean([len(item) for item in preprocessed_corpus]))
    print('Average number of words per BoW document: ',np.mean([len(corpus[i]) for i in range(len(corpus))]))

    return dictionary, corpus

In [ ]:
working_dict, working_corpus = bag_of_word_processing(preprocessed_corpus, 10, 0.7)

## Building the topic model

In [ ]:
from gensim.models import CoherenceModel, ldamulticore

def calculate_scores(dictionary, corpus,  texts, limit, start=2, step=3):
    """
    Compute c_v coherence for a wide range of topic numbers.
    Adapted from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    method = c_v or u_mass
    texts : List of input texts (doc_clean)
    limit : Max num of topics
    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    graphical outputs
    """
    coherence_dict = dict()

    for num_topics in range(start, limit, step):
        model = ldamulticore.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=2)
        coherencemodel1 = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_dict[num_topics] = coherencemodel1.get_coherence()

    coherence_df = pd.DataFrame(pd.Series(coherence_dict)).reset_index()

    coherence_df.columns = ['Num_topics','Coherence_score']

    # Show graph
    fig, ax = plt.subplots(figsize=(12,10))
    ax.plot(coherence_df['Num_topics'], coherence_df['Coherence_score'])
    ax.set_xlabel("No. of topics", fontweight='bold')
    ax.set_ylabel("Cv Coherence score", fontweight='bold')
    ax.axvline(coherence_df[coherence_df['Coherence_score'] == coherence_df['Coherence_score'].max()]['Num_topics'].tolist(), color='red')

    return coherence_df

In [ ]:
calculate_scores(working_dict, working_corpus, preprocessed_corpus, limit=100)